# Deep Video Analytics tutorial

In [ ]:
import os, requests
message = """Please note that this notebook MUST be run INSIDE the container, 
To use this notebook run "start_tutorial.py" in your host OS. And then follow instructions.
Once you have gone through tutorial you can use DVA client from anywhere by connecting to a 
DVA server with auth token. 
"""

if os.path.exists('/root/DVA'):
    if requests.get('http://localhost').ok:
        print "Running correctly inside container"
    else:
        print "Could not connect to localhost, seems DVA server is not yet running"
else: 
    raise SystemError(message)

### This tutorial uses REST API endpoint wrapped in a DVA python client.
### Install DVA Client  

In [ ]:
!cd ../../client && python setup.py install

In [ ]:
import dvaclient
import logging, json

## Using Deep Video Analytics python client

####  Following command generate & copy a REST API token, it can also be found at /security page from the sidebar in DVA UI

In [ ]:
!cd ../../server/scripts/ && python generate_testing_token.py

In [ ]:
ctx = dvaclient.context.DVAContext(server='http://localhost/', 
                                   token=json.load(file('../../server/scripts/creds.json'))['token'])

## REST API vs DVAPQL

Deep Video Analytics uses REST API (GET, LIST, but **NOT POST**) for exposing state of the system along metadata and urls to raw contents such as videos and images. **/queries is the only endpoint that supports POST requests** for mutating the state of the system. The user submits a DVAPQL query for loading videos/datasets, launching processing jobs, importing models and performing visual search. 

### List videos

In [ ]:
for v in ctx.list_videos():
    print "{name} with ID: {vid}".format(name=v['name'],vid=v['id'])

### List configured indexing, detection and analysis models

In [ ]:
for v in ctx.list_models():
    print v['name'], v['model_type']

## List events

In [ ]:
_ = ctx.list_events(verbose=True)

## Process a video

In [ ]:
video_processing_query = dvaclient.query.ProcessVideoURL(name="spectre",url="https://www.youtube.com/watch?v=ashLaclKCik")
video_processing_query.execute(ctx)

##### Optionally you can "wait" on it to complete,

In [ ]:
video_processing_query.wait(verbose=True)

## Finding similar images

### Query image

In [ ]:
leo_query_image = 'figures/query_leo.png'

### Get configured retrievers and indexers

In [ ]:
indexers = {r['name']:r for r in ctx.list_models() if r['model_type'] == 'I'}
retrievers = {r['name']:r for r in ctx.list_retrievers()}

### Make a query using inception retriever and inception indexer

In [ ]:
inception_indexer_pk = indexers['inception']['id']
inception_retriever_pk = retrievers['inception']['id']
q = dvaclient.query.FindSimilarImages(leo_query_image,indexer_pk=inception_indexer_pk,retriever_pk=inception_retriever_pk)
q.execute(context=ctx)
q.wait()
q.gather_search_results()

### Visualize results

In [ ]:
for r in q.search_results:
    r.visualize()

In [ ]:
q = dvaclient.query.FindSimilarImages('figures/query_car.png',indexer_pk=inception_indexer_pk,retriever_pk=inception_retriever_pk)
q.execute(context=ctx)
q.wait()
q.gather_search_results()
for r in q.search_results:
    r.visualize()